<a href="https://colab.research.google.com/github/SalvatoreRomano1/net2020/blob/main/Post_Group/Topics-Comments/Positive_Comments_Topics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Add attributes to node

In [ ]:
simport math
def create_attributes_dict(pandas_df, index_shift):
    #attributes=["p_PARTITO","p_politician","p_text","p_favoriteCount","p_shareCount","p_replyCount","p_numComments","p_rating","p_topic","p_campagna","p_targe1-2"]
    attributes=["p_PARTITO","p_politician","p_favoriteCount","p_shareCount","p_replyCount","p_numComments","p_rating","p_topic","p_campagna"]
    att_dict = {} #final dict to be returned
    for index in pandas_df.index:
    #for index, row in pandas_df.iterrows():
        #if index not in att_dict: #check seems to make issues...
        temp_dict={}
        for att in attributes:
            #print(att)
            #print(pandas_df[att][0])
            #print(index)
            #print(row)
            #print(pandas_df[att][index])

            #we need to delete the "nan" values
            if(isinstance(pandas_df[att][index], float)):
                if(not math.isnan(pandas_df[att][index])):
                    temp_dict[att]=pandas_df[att][index]
            else:
                temp_dict[att]=pandas_df[att][index]
        temp_dict["label"]=index+shift_size
        temp_dict["type"]="post"
        att_dict[index+index_shift]=temp_dict.copy() #I'm afraid if shallow copy (not sure if usefull though)
    return att_dict

In [ ]:
def add_pol_party_att(pandas_df, parties, politicians): #parties & pol= lists of parties and politicians
    att_dict = {}
    att_dict[0]={"label":"root", "type":"root"}
    for i in range(len(parties)):
        att_dict[i+1]={"label":parties[i], "type":"party"}
    for i in range(len(politicians)):
        att_dict[i+len(parties)+1] = {"label":politicians[i], "type":"politician"}
    return att_dict

In [ ]:
import networkx as nx
#Sandbox on trunc graph
G_trunc = nx.from_numpy_matrix(m_trunc)
attrs = {0: {"p_PARTITO": df_post['p_PARTITO'][0], "p_politician": df_post['p_politician'][0]}}
nx.set_node_attributes(G_trunc, attrs)
G_trunc.nodes(data=True)

#try the function
truc_df=df_post[:20]
test_att=create_attributes_dict(truc_df,shift_size)
#test_att

nx.set_node_attributes(G_trunc, test_att)
#G_trunc.nodes(data=True) #to visualize node attributes
nx.write_graphml(G_trunc, "test_attributes_networkx_export.graphml")

NameError: ignored

In [ ]:
#Apply function on real dataframe
import networkx as nx
G = nx.from_numpy_matrix(m)
attributes_dict=create_attributes_dict(df_post,shift_size)
missing_attributes_dict=add_pol_party_att(df_post,parties,pol)
attributes_dict.update(missing_attributes_dict)
nx.set_node_attributes(G, attributes_dict)
nx.write_graphml(G, "test_attributes_networkx_export.graphml")

## Sandbox

In [ ]:
#Different ways to itterate over dataframe
"""
for index, row in df_post.iterrows():
    print(index)
for i in df_post.index: 
    print(i) """

In [ ]:
#Dealing with "nan" values
print(parties)
df_post.loc[df_post['p_PARTITO'].isna()].shape
df_post.iloc[(106-shift_size):(111-shift_size),:]
type(df_post["p_PARTITO"][22])

## some checks

In [ ]:
#matrix symmetry
print((m.transpose() == m).all())

In [ ]:
#check number of values on first row
first_row=m[0,:] #type numpy.matrix
first_row = np.squeeze(np.asarray(first_row)) #type array
sum(filter(lambda x: x>0, first_row))
#10104 is good! (posts+root)

In [ ]:
#check number of values on first col
first_col=m[:,0] #type numpy.matrix
first_col = np.squeeze(np.asarray(first_col)) #type array
sum(filter(lambda x: x>0, first_col))
#10104 is good! (posts+root)

In [ ]:
#edges labels
G_trunc.edges(data=True)

## start from here

In [ ]:
import pandas as pd 
import numpy as np 

import statsmodels
import statsmodels.api as sm
import scipy.stats as stats

import matplotlib.pyplot as plt

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# import the csv file with JUST the politicians post
comDB = pd.read_csv(r"/content/gdrive/MyDrive/database/com_liwc.csv", sep='\t', engine='python')

df = pd.DataFrame(data=comDB)

pd.set_option('display.max_columns', None)
df

,Origin_file_order,Site,p_id,dateCreated,p_politician,p_gender,p_GRUPPO_PE,p_LISTA,p_PARTITO,p_governo,p_dx_sx,p_CIRCOSCRIZIONE,p_text,p_favoriteCount,p_shareCount,p_replyCount,p_replyEval,p_numComments,p_numFakeTags,p_rating,p_category,p_topic,p_campagna,p_camapagna2,Target1,Target2,p_targe1-2,target1_s-p,target1_pol,c_text,c_level,c_replyToUser,c_replyToText,c_rating,c_rating3,c_ratingCivile,c_ratingPosNeg,c_category,Unnamed: 38,c_topic,isPersonal,c_WC,c_WPS,c_Sixltr,c_Dic,c_pronomi,c_Io,c_Noi,c_Se,c_Tu,c_Altri,c_Negazio,c_Consen,c_Articol,c_Prepos,c_Numero,c_Affett,c_Sen_Pos,c_Emo_Pos,c_Ottimis,c_Emo_Neg,c_Ansia,c_Rabbia,c_Tristez,c_Mec_Cog,c_Causa,c_Intros,c_Discrep,c_Inibiz,c_possib,c_Certez,c_Proc_Sen,c_Vista,c_Udito,c_Sentim,c_Social,c_Comm,c_Rif_gen,c_amici,c_Famigl,c_Umano,c_Tempo,c_Passato,c_Present,c_Futuro,c_Spazio,c_Sopra,c_Sotto,c_Inclusi,c_Esclusi,c_Movimen,c_Occupaz,c_Scuola,c_Lavoro,c_Raggiun,c_Svago,c_Casa,c_Sport,c_TV_it,c_Musica,c_Soldi,c_Metafis,c_religio,c_Morte,c_Fisico,c_Corpo,c_Sesso,c_Mangiare,c_Dormire,c_Cura_cor,c_parolac,c_Non_flu,c_riempiti,c_Voi,c_Lui_lei,c_Loro,c_Condizio,c_Transiti,c_P_pass,c_gerundio,c_Passivo,c_Essere,c_Avere,c_Formale,c_Io_Ver,c_Tu_Verbo,c_Lui_Verb,c_Noi_Verb,c_Voi_Verb,c_Loro_Ver,c_AllPunc,c_Period,c_Comma,c_Colon,c_SemiC,c_Qmark,c_Exclam,c_Dash,c_Quote,c_Apostro,c_Parenth,c_OtherP,p_WC,p_WPS,p_Sixltr,p_Dic,p_pronomi,p_Io,p_Noi,p_Se,p_Tu,p_Altri,p_Negazio,p_Consen,p_Articol,p_Prepos,p_Numero,p_Affett,p_Sen_Pos,p_Emo_Pos,p_Ottimis,p_Emo_Neg,p_Ansia,p_Rabbia,p_Tristez,p_Mec_Cog,p_Causa,Ip_ntros,p_Discrep,Ip_nibiz,p_possib,p_Certez,p_Proc_Sen,p_Vista,p_Udito,p_Sentim,p_Social,p_Comm,p_Rif_gen,p_amici,p_Famigl,p_Umano,p_Tempo,p_Passato,p_Present,p_Futuro,p_Spazio,p_Sopra,p_Sotto,Ip_nclusi,p_Esclusi,p_Movimen,p_Occupaz,p_Scuola,p_Lavoro,p_Raggiun,p_Svago,p_Casa,p_Sport,p_TV_it,p_Musica,p_Soldi,p_Metafis,p_religio,p_Morte,p_Fisico,p_Corpo,p_Sesso,p_Mangiare,p_Dormire,p_Cura_cor,p_parolac,p_Non_flu,p_riempiti,p_Voi,p_Lui_lei,p_Loro,p_Condizio,p_Transiti,p_P_pass,p_gerundio,p_Passivo,p_Essere,p_Avere,p_Formale,p_Io_Ver,p_Tu_Verbo,p_Lui_Verb,p_Noi_Verb,p_Voi_Verb,p_Loro_Ver,p_AllPunc,p_Period,p_Comma,p_Colon,p_SemiC,p_Qmark,p_Exclam,p_Dash,p_Quote,p_Apostro,p_Parenth,p_OtherP
0,30126,FB,96844400700_10157493758850701,2019-04-23T10:33:37Z,MARCELLO GEMMATO,M,PPE,FDI,FDI,opposizione,destra,s,Se i musulmani pensano di portare la guerra sa...,270,80,57,33,1729,0,problematico,Rifugiati Musulmani,religioni europa,Comparativa,Neg-comp,Categoria di persone,NaN,Categoria di persone,gruppo,non politico,C'è poco da dire questa è gente che la guerra ...,1,FRATELLIDITALIA.PUGLIA,Se i musulmani pensano di portare la guerra sa...,problematico,probl-hate,incivile,negativo,Rifugiati Musulmani,NaN,Other,Other,4,2.0,50.00,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.00,0.0,0.00,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0,0.00,0.00,0,0.00,0.0,0.00,0.00,0.0,0.0,125.00,100.00,0.00,0.0,0.0,0.00,25.00,0.0,0.0,0.00,0.0,0.00,6,6.00,33.33,33.33,0.00,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.00,33.33,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0,0.00,0.00,0,0.00,0.0,0.00,0.0,0.0,0.0,16.67,16.67,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00
1,30226,FB,96844400700_10157493758850701,2019-04-23T10:33:37Z,MARCELLO GEMMATO,M,PPE,FDI,FDI,opposizione,destra,s,Se i musulmani pensano di portare la guerra sa...,270,80,57,33,1729,0,problematico,Rifugiati Musulmani,religioni europa,Comparativa,Neg-comp,Categoria di persone,NaN,Categoria di persone,gruppo,non p

In [ ]:
com_type = df['c_rating'].unique().tolist()
print(com_type)

['problematico', 'positivo', 'negativo', 'hate', 'ambiguo']


In [ ]:
df_problematico = df[df['c_rating'] == 'problematico']
df_positivo = df[df['c_rating'] == 'positivo']
df_negativo = df[df['c_rating'] == 'negativo']
df_hate = df[df['c_rating'] == 'hate']
df_ambiguo = df[df['c_rating'] == 'ambiguo']

In [ ]:
pd.set_option('display.max_columns', None)
#df_problematico
#df_hate
df_positivo

,Origin_file_order,Site,p_id,dateCreated,p_politician,p_gender,p_GRUPPO_PE,p_LISTA,p_PARTITO,p_governo,p_dx_sx,p_CIRCOSCRIZIONE,p_text,p_favoriteCount,p_shareCount,p_replyCount,p_replyEval,p_numComments,p_numFakeTags,p_rating,p_category,p_topic,p_campagna,p_camapagna2,Target1,Target2,p_targe1-2,target1_s-p,target1_pol,c_text,c_level,c_replyToUser,c_replyToText,c_rating,c_rating3,c_ratingCivile,c_ratingPosNeg,c_category,Unnamed: 38,c_topic,isPersonal,c_WC,c_WPS,c_Sixltr,c_Dic,c_pronomi,c_Io,c_Noi,c_Se,c_Tu,c_Altri,c_Negazio,c_Consen,c_Articol,c_Prepos,c_Numero,c_Affett,c_Sen_Pos,c_Emo_Pos,c_Ottimis,c_Emo_Neg,c_Ansia,c_Rabbia,c_Tristez,c_Mec_Cog,c_Causa,c_Intros,c_Discrep,c_Inibiz,c_possib,c_Certez,c_Proc_Sen,c_Vista,c_Udito,c_Sentim,c_Social,c_Comm,c_Rif_gen,c_amici,c_Famigl,c_Umano,c_Tempo,c_Passato,c_Present,c_Futuro,c_Spazio,c_Sopra,c_Sotto,c_Inclusi,c_Esclusi,c_Movimen,c_Occupaz,c_Scuola,c_Lavoro,c_Raggiun,c_Svago,c_Casa,c_Sport,c_TV_it,c_Musica,c_Soldi,c_Metafis,c_religio,c_Morte,c_Fisico,c_Corpo,c_Sesso,c_Mangiare,c_Dormire,c_Cura_cor,c_parolac,c_Non_flu,c_riempiti,c_Voi,c_Lui_lei,c_Loro,c_Condizio,c_Transiti,c_P_pass,c_gerundio,c_Passivo,c_Essere,c_Avere,c_Formale,c_Io_Ver,c_Tu_Verbo,c_Lui_Verb,c_Noi_Verb,c_Voi_Verb,c_Loro_Ver,c_AllPunc,c_Period,c_Comma,c_Colon,c_SemiC,c_Qmark,c_Exclam,c_Dash,c_Quote,c_Apostro,c_Parenth,c_OtherP,p_WC,p_WPS,p_Sixltr,p_Dic,p_pronomi,p_Io,p_Noi,p_Se,p_Tu,p_Altri,p_Negazio,p_Consen,p_Articol,p_Prepos,p_Numero,p_Affett,p_Sen_Pos,p_Emo_Pos,p_Ottimis,p_Emo_Neg,p_Ansia,p_Rabbia,p_Tristez,p_Mec_Cog,p_Causa,Ip_ntros,p_Discrep,Ip_nibiz,p_possib,p_Certez,p_Proc_Sen,p_Vista,p_Udito,p_Sentim,p_Social,p_Comm,p_Rif_gen,p_amici,p_Famigl,p_Umano,p_Tempo,p_Passato,p_Present,p_Futuro,p_Spazio,p_Sopra,p_Sotto,Ip_nclusi,p_Esclusi,p_Movimen,p_Occupaz,p_Scuola,p_Lavoro,p_Raggiun,p_Svago,p_Casa,p_Sport,p_TV_it,p_Musica,p_Soldi,p_Metafis,p_religio,p_Morte,p_Fisico,p_Corpo,p_Sesso,p_Mangiare,p_Dormire,p_Cura_cor,p_parolac,p_Non_flu,p_riempiti,p_Voi,p_Lui_lei,p_Loro,p_Condizio,p_Transiti,p_P_pass,p_gerundio,p_Passivo,p_Essere,p_Avere,p_Formale,p_Io_Ver,p_Tu_Verbo,p_Lui_Verb,p_Noi_Verb,p_Voi_Verb,p_Loro_Ver,p_AllPunc,p_Period,p_Comma,p_Colon,p_SemiC,p_Qmark,p_Exclam,p_Dash,p_Quote,p_Apostro,p_Parenth,p_OtherP
370,76380,FB,96844400700_10157503425970701,2019-04-26T04:17:54Z,MARCELLO GEMMATO,M,PPE,FDI,FDI,opposizione,destra,s,🔵 PRIMA CASA DIRITTO INVIOLABILE 🇮🇹\n\nPer Fr...,210,52,8,4,1729,0,positivo,None,Other,Comparativa,Neg-comp,Categoria di persone,NaN,Categoria di persone,gruppo,non politico,STRANO che NESSUNO DELLE “SINISTRE” abbia MAI...,1,FRATELLIDITALIA.PUGLIA,🔵 PRIMA CASA DIRITTO INVIOLABILE 🇮🇹\n\nPer Fr...,positivo,positivo,civile,positivo,None,NaN,altroPolitico,Other_pol,6,6.00,16.67,16.67,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.00,0.00,0.0,0.0,0.0,0.0,16.67,16.67,0.00,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0,0.0,0.00,0,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,10,5.00,20.00,40.00,0.00,0.0,0.00,0.0,0.0,0.0,0.00,0.0,10.00,20.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.00,0.0,0.00,0.00,0.00,0.00,0.0,0.0,10.00,0.0,0.00,0.0,10.00,0.00,0.00,0.0,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0,0.0,0.00,0,0.0,0.00,0.00,0.00,0.0,0.00,40.00,0.00,0.00,0.00,0.0,0.0,10.00,0.0,0.00,20.00,0.0,10.00
371,76384,FB,96844400700_10157503425970701,2019-04-26T04:17:54Z,MARCELLO GEMMATO,M,PPE,FDI,FDI,opposizione,destra,s,🔵 PRIMA CASA DIRITTO INVIOLABILE 🇮🇹\n\nPer Fr...,210,52,8,4,1729,0,positivo,None,Other,Comparativa,Neg-comp,Categoria di persone,NaN,Categoria di persone,gruppo,non politico,"MA QUELLI SONO VECCHI TENTATIVI, DI SEMPRE DEL...",1,FRATEL

In [ ]:
#df_problematico = df_problematico.reset_index()
#df_problematico = df_problematico.drop('index',axis = 1)

df_positivo = df_positivo.reset_index()
df_positivo = df_positivo.drop('index',axis = 1)

In [ ]:
#df_problematico
#df_hate
df_positivo

,Origin_file_order,Site,p_id,dateCreated,p_politician,p_gender,p_GRUPPO_PE,p_LISTA,p_PARTITO,p_governo,p_dx_sx,p_CIRCOSCRIZIONE,p_text,p_favoriteCount,p_shareCount,p_replyCount,p_replyEval,p_numComments,p_numFakeTags,p_rating,p_category,p_topic,p_campagna,p_camapagna2,Target1,Target2,p_targe1-2,target1_s-p,target1_pol,c_text,c_level,c_replyToUser,c_replyToText,c_rating,c_rating3,c_ratingCivile,c_ratingPosNeg,c_category,Unnamed: 38,c_topic,isPersonal,c_WC,c_WPS,c_Sixltr,c_Dic,c_pronomi,c_Io,c_Noi,c_Se,c_Tu,c_Altri,c_Negazio,c_Consen,c_Articol,c_Prepos,c_Numero,c_Affett,c_Sen_Pos,c_Emo_Pos,c_Ottimis,c_Emo_Neg,c_Ansia,c_Rabbia,c_Tristez,c_Mec_Cog,c_Causa,c_Intros,c_Discrep,c_Inibiz,c_possib,c_Certez,c_Proc_Sen,c_Vista,c_Udito,c_Sentim,c_Social,c_Comm,c_Rif_gen,c_amici,c_Famigl,c_Umano,c_Tempo,c_Passato,c_Present,c_Futuro,c_Spazio,c_Sopra,c_Sotto,c_Inclusi,c_Esclusi,c_Movimen,c_Occupaz,c_Scuola,c_Lavoro,c_Raggiun,c_Svago,c_Casa,c_Sport,c_TV_it,c_Musica,c_Soldi,c_Metafis,c_religio,c_Morte,c_Fisico,c_Corpo,c_Sesso,c_Mangiare,c_Dormire,c_Cura_cor,c_parolac,c_Non_flu,c_riempiti,c_Voi,c_Lui_lei,c_Loro,c_Condizio,c_Transiti,c_P_pass,c_gerundio,c_Passivo,c_Essere,c_Avere,c_Formale,c_Io_Ver,c_Tu_Verbo,c_Lui_Verb,c_Noi_Verb,c_Voi_Verb,c_Loro_Ver,c_AllPunc,c_Period,c_Comma,c_Colon,c_SemiC,c_Qmark,c_Exclam,c_Dash,c_Quote,c_Apostro,c_Parenth,c_OtherP,p_WC,p_WPS,p_Sixltr,p_Dic,p_pronomi,p_Io,p_Noi,p_Se,p_Tu,p_Altri,p_Negazio,p_Consen,p_Articol,p_Prepos,p_Numero,p_Affett,p_Sen_Pos,p_Emo_Pos,p_Ottimis,p_Emo_Neg,p_Ansia,p_Rabbia,p_Tristez,p_Mec_Cog,p_Causa,Ip_ntros,p_Discrep,Ip_nibiz,p_possib,p_Certez,p_Proc_Sen,p_Vista,p_Udito,p_Sentim,p_Social,p_Comm,p_Rif_gen,p_amici,p_Famigl,p_Umano,p_Tempo,p_Passato,p_Present,p_Futuro,p_Spazio,p_Sopra,p_Sotto,Ip_nclusi,p_Esclusi,p_Movimen,p_Occupaz,p_Scuola,p_Lavoro,p_Raggiun,p_Svago,p_Casa,p_Sport,p_TV_it,p_Musica,p_Soldi,p_Metafis,p_religio,p_Morte,p_Fisico,p_Corpo,p_Sesso,p_Mangiare,p_Dormire,p_Cura_cor,p_parolac,p_Non_flu,p_riempiti,p_Voi,p_Lui_lei,p_Loro,p_Condizio,p_Transiti,p_P_pass,p_gerundio,p_Passivo,p_Essere,p_Avere,p_Formale,p_Io_Ver,p_Tu_Verbo,p_Lui_Verb,p_Noi_Verb,p_Voi_Verb,p_Loro_Ver,p_AllPunc,p_Period,p_Comma,p_Colon,p_SemiC,p_Qmark,p_Exclam,p_Dash,p_Quote,p_Apostro,p_Parenth,p_OtherP
0,76380,FB,96844400700_10157503425970701,2019-04-26T04:17:54Z,MARCELLO GEMMATO,M,PPE,FDI,FDI,opposizione,destra,s,🔵 PRIMA CASA DIRITTO INVIOLABILE 🇮🇹\n\nPer Fr...,210,52,8,4,1729,0,positivo,None,Other,Comparativa,Neg-comp,Categoria di persone,NaN,Categoria di persone,gruppo,non politico,STRANO che NESSUNO DELLE “SINISTRE” abbia MAI...,1,FRATELLIDITALIA.PUGLIA,🔵 PRIMA CASA DIRITTO INVIOLABILE 🇮🇹\n\nPer Fr...,positivo,positivo,civile,positivo,None,NaN,altroPolitico,Other_pol,6,6.00,16.67,16.67,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.00,0.00,0.0,0.0,0.0,0.0,16.67,16.67,0.00,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0,0.0,0.00,0,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,10,5.00,20.00,40.00,0.00,0.0,0.00,0.0,0.0,0.0,0.00,0.0,10.00,20.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.00,0.0,0.00,0.00,0.00,0.00,0.0,0.0,10.00,0.0,0.00,0.0,10.00,0.00,0.00,0.0,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0,0.0,0.00,0,0.0,0.00,0.00,0.00,0.0,0.00,40.00,0.00,0.00,0.00,0.0,0.0,10.00,0.0,0.00,20.00,0.0,10.00
1,76384,FB,96844400700_10157503425970701,2019-04-26T04:17:54Z,MARCELLO GEMMATO,M,PPE,FDI,FDI,opposizione,destra,s,🔵 PRIMA CASA DIRITTO INVIOLABILE 🇮🇹\n\nPer Fr...,210,52,8,4,1729,0,positivo,None,Other,Comparativa,Neg-comp,Categoria di persone,NaN,Categoria di persone,gruppo,non politico,"MA QUELLI SONO VECCHI TENTATIVI, DI SEMPRE DEL...",1,FRATELLIDI

biadjacency matrix topic

In [ ]:
'''
topic=df_problematico['c_topic'].unique().tolist()
#print(len(topic))

topics = set()
for i in range(len(topic)):
    temp = topic[i]
    temp = temp.split(' ')

    for j in range(len(temp)):
        topics.add(temp[j])

topics = list(topics)

SyntaxError: ignored

In [ ]:
topic=df_positivo['c_topic'].unique().tolist()
#print(len(topic))

topics = set()
for i in range(len(topic)):
    temp = topic[i]
    temp = temp.split(' ')

    for j in range(len(temp)):
        topics.add(temp[j])

topics = list(topics)

In [ ]:
'''#biadjacency matrix 
size_df= len(df_problematico)
#matrix = np.zeros((size_df, len(topics))) #rows: comm, columns: topic
matrix = np.zeros((len(topics),size_df)) 
print(matrix.shape)

for i in range(size_df):
  #print('iter:'+ str(i))
  #comm_search = df_problematico['p_id'][i]
  topic_search = df_problematico['c_topic'][i]
  topic_search= list(topic_search.split(" ")) 
  #print('search:'+str(topic_search))
  for j in range(len(topic_search)):
    temp = topic_search[j]
    index = topics.index(temp)
    #print('index:'+ str(index))
    #matrix[i][index]+=1 
    matrix[index][i]+=1 
#print(matrix)'''


In [ ]:
import networkx as nx
from scipy import sparse 


In [ ]:
'''
pol =  df_problematico['p_politician'].unique().tolist()

size_df= len(df_problematico)
#matrix = np.zeros((size_df, len(topics))) #rows: comm, columns: topic
matrix = np.zeros((len(topics)+len(pol),size_df)) 
print(matrix.shape)
shift = len(topics)

for i in range(size_df):
  #print('iter:'+ str(i))
  #comm_search = df_problematico['p_id'][i]
  topic_search = df_problematico['c_topic'][i]
  topic_search= list(topic_search.split(" ")) 
  #print('search:'+str(topic_search))
  for j in range(len(topic_search)):
    temp = topic_search[j]
    index = topics.index(temp)
    #print('index:'+ str(index))
    #matrix[i][index]+=1 
    matrix[index][i]+=1 
for i in range(size_df):
  pol_search =  df_problematico['p_politician'][i]
  pol_index = pol.index(pol_search)
  matrix[pol_index+shift][i]+=1

#print(matrix)

In [ ]:
pol =  df_positivo['p_politician'].unique().tolist()

size_df= len(df_positivo)
#matrix = np.zeros((size_df, len(topics))) #rows: comm, columns: topic
matrix = np.zeros((len(topics)+len(pol),size_df)) 
print(matrix.shape)
shift = len(topics)

for i in range(size_df):
  #print('iter:'+ str(i))
  #comm_search = df_problematico['p_id'][i]
  topic_search = df_positivo['c_topic'][i]
  topic_search= list(topic_search.split(" ")) 
  #print('search:'+str(topic_search))
  for j in range(len(topic_search)):
    temp = topic_search[j]
    index = topics.index(temp)
    #print('index:'+ str(index))
    #matrix[i][index]+=1 
    matrix[index][i]+=1 
for i in range(size_df):
  pol_search =  df_positivo['p_politician'][i]
  pol_index = pol.index(pol_search)
  matrix[pol_index+shift][i]+=1

#print(matrix)

(90, 41954)


In [ ]:
matrix_sparse = sparse.csr_matrix(matrix)

In [ ]:
G_biad_topic = nx.bipartite.from_biadjacency_matrix(matrix_sparse) #bipartite matrix graph

In [ ]:
nodes = []
for i in range(size_df):
  nodes.append(i)
projected_withpol = nx.bipartite.projection.projected_graph(G_biad_topic, nodes)

nx.write_graphml(projected_withpol, "projected_topic_pol_positivo.graphml")

NameError: ignored

In [ ]:
print(nx.number_of_edges(G_biad_topic))

In [ ]:
nx.write_graphml(G_biad_topic, "bipartite_pol_topic_positivo.graphml")

In [ ]:
nx.write_weighted_edgelist(G_biad_topic, "edgelist_weighted_pol_topic_positivo.csv",delimiter=",")
!sed -i.bak 1i"Source,Target,Weight" edgelist_weighted_pol_topic_positivo.csv

NameError: ignored

In [ ]:
node= pd.DataFrame(topics+pol,columns =['Label'])

node['type']="politician"
node.iloc[0:14, node.columns.get_loc('type')] = 'topic'
l = [x for x in range(len(topics)+len(pol))]

node['Id'] = l

In [ ]:
node

In [ ]:
attribute = df_hate[['c_topic',"p_PARTITO", 'p_politician']]
attribute.reset_index(inplace=True)
#attribute[0] = attribute['index']
l = [x for x in range(len(topics)+len(pol),len(attribute)+len(topics)+len(pol))]
attribute['Id'] = l
attribute['type'] = 'comment'
attribute.rename(columns={0: 'Label', 'p_PARTITO': 'partito', 'p_politician': 'politician'}, inplace=True)




#attribute.to_csv("nodelist_pol_topic.csv",index=False)

In [ ]:
attribute

In [ ]:
result = node.append(attribute)

In [ ]:
result.drop(['index'], axis=1)

In [ ]:
result.to_csv("nodelist_pol_topic_positivo.csv",index=False)